In [3]:
# General Imports
import pandas as pd
import sys
import os
import re
import pickle

# Map src directory
root_dir = os.path.abspath(os.path.join(os.getcwd(), "../."))
print("Root Directory: ", root_dir)
src_dir = os.path.join(root_dir,"src","complete_collision_recorder")
print("Src Directory: ", src_dir)
sys.path.append(src_dir)
data_dir = os.path.join(root_dir,"data")
print("Data Directory: ", data_dir)
models_dir = os.path.join(root_dir,"models")
print("Models Directory: ", data_dir)

# Import src modules
import complete_collision as cc
import utils as u

Root Directory:  /media/dzielinski06/HDD1/AI894 - Capstone/Complete Collision Recorder
Src Directory:  /media/dzielinski06/HDD1/AI894 - Capstone/Complete Collision Recorder/src/complete_collision_recorder
Data Directory:  /media/dzielinski06/HDD1/AI894 - Capstone/Complete Collision Recorder/data
Models Directory:  /media/dzielinski06/HDD1/AI894 - Capstone/Complete Collision Recorder/data


In [5]:
# root_dir, src_dir, data_dir, models_dir = u.map_project_directories()


In [4]:
training_df = pd.read_csv(os.path.join(data_dir, "processed", "training_df.csv"))
training_df['concatenated_text'] = training_df.apply(u.concatenate_texts, axis=1)
u.clean_text(training_df, 'concatenated_text')

,CAD_ID,CAD_TEXT,OH1_TEXT,BIKE_CLE_TEXT,CRASH_SEVERITY_CD,INTERSECTION_IND,U1_PRECRASH_ACTION_CD,U1_TURN_CD,U1_TYPE_OF_UNIT_CD,U2_PRECRASH_ACTION_CD,U2_TURN_CD,U2_TYPE_OF_UNIT_CD,concatenated_text
0,202000000930,CALL COMMENTS COMMENTS FROM INTERGAPH LOI SEAR...,NaN,A driver struck an adult crossing an intersect...,3.0,Y,99.0,NT,1.0,1.0,NT,1.0,CALL COMMENTS COMMENTS FROM INTERGAPH LOI SEAR...
1,202000001607,NaN,"WHILE UNIT1 WAS TURNING ONTO W44_, PEDESTRIAN ...",A driver of a commercial truck turned left and...,5.0,Y,1.0,NT,1.0,11.0,NT,1.0,POLICE NARRATIVE \n\n WHILE UNIT1 WAS TURNING ...
2,202000002596,CALL COMMENTS COMMENTS FROM INTERGAPH LOI SEAR...,NaN,A driver struck a person. The crash caused inj...,2.0,Y,6.0,LT,3.0,1.0,NT,1.0,CALL COMMENTS COMMENTS FROM INTERGAPH LOI SEAR...
3,202000002644,CALL COMMENTS COMMENTS FROM INTERGAPH LOI SEAR...,UNKNOWN EVENTS,A driver turned left and struck and killed an ...,5.0,Y,1.0,NT,99.0,11.0,NT,1.0,CALL COMMENTS COMMENTS FROM INTERGAPH LOI SEAR...
4,202000003457,CALL COMMENTS COMMENTS FROM INTERGAPH LOI SEAR...,NaN,A driver struck an adult. The crash caused inj...,5.0,Y,1.0,NT,1.0,NaN,NaN,NaN,CALL COMMENTS COMMENTS FROM INTERGAPH LOI SEAR...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2170,202300386537,CALL COMMENTS COMMENTS FROM INTERGAPH LOI SEAR...,UNKNOWN EVENTS,A driver of an SUV turned left and struck an a...,4.0,Y,6.0,LT,99.0,14.0,NT,23.0,CALL COMMENTS COMMENTS FROM INTERGAPH LOI SEAR...
2171,202300386861,CALL COMMENTS COMMENTS FROM INTERGAPH PED STRU...,NaN,A driver of a transit bus struck a person.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CALL COMMENTS COMMENTS FROM INTERGAPH PED STRU...
2172,202300387472,CALL COMMENTS COMMENTS FROM INTERGAPH LOI SEAR...,NaN,A driver struck a person.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CALL COMMENTS COMMENTS FROM INTERGAPH LOI SEAR...
2173,202300388004,CALL COMMENTS COMMENTS FROM INTERGAPH LOI SEAR...,NaN,A driver struck a person on a bicycle. The dri...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CALL COMMENTS COMMENTS FROM INTERGAPH LOI SEAR...


In [18]:
# Load the severity model
with open(os.path.join(models_dir,'severity.pkl'), 'rb') as f:
    severity_model = pickle.load(f)

training_df['SEVERITY_PRED'] = severity_model.predict(training_df['concatenated_text'])
training_df['SEVERITY_PRED'].value_counts()


In [33]:
# # Preprocessing Packages
# import nltk
# from nltk.stem import PorterStemmer
# from sklearn.feature_extraction.text import TfidfVectorizer
# from imblearn.over_sampling import SMOTE, SVMSMOTE, KMeansSMOTE, ADASYN, BorderlineSMOTE, RandomOverSampler
# from sklearn.preprocessing import StandardScaler, FunctionTransformer, MaxAbsScaler
# from sklearn.decomposition import PCA, TruncatedSVD
# from sklearn.cluster import KMeans
# ps = PorterStemmer()
# stopWords = set(nltk.corpus.stopwords.words('english'))
# charfilter = re.compile('[a-zA-Z]+')

# def CCR_Tokenizer(text):
#     words = map(lambda word: word.lower(), nltk.word_tokenize(text))
#     words = [word for word in words if word not in stopWords]
#     tokens = (list(map(lambda token: ps.stem(token), words)))
#     ntokens = list(filter(lambda token: charfilter.match(token), tokens))
#     return ntokens

# vec = TfidfVectorizer(tokenizer=CCR_Tokenizer, 
#                       norm='l2', 
#                       vocabulary=None,
#                       min_df=0.05,
#                       max_df=0.9,
#                       max_features=500,
#                       )

In [40]:
import pickle
import numpy as np
from sklearn.svm import LinearSVC

# Load the pipeline from a pickle file
with open(os.path.join(models_dir,'severity.pkl'), 'rb') as f:
    pipeline = pickle.load(f)

# Inspect the pipeline to confirm step names
print("Pipeline steps:", pipeline.named_steps)

svc_model = pipeline.named_steps['classifier']  

# Ensure the model is a LinearSVC
if isinstance(svc_model, LinearSVC):
    # Raw input text (replace with your actual data)
    raw_input = training_df['concatenated_text'].iloc[2]  # Example: third row of the dataset

    # Pass the raw input through the entire pipeline
    transformed_instance = pipeline[:-1].transform([raw_input])  # Exclude the classifier step

    # Get the vectorizer from the pipeline
    vectorizer = pipeline.named_steps['vec']  # Replace 'vectorizer' with your actual vectorizer step name

    # Predict the class
    predicted_class = svc_model.predict(transformed_instance)[0]

    # Feature contributions for the predicted class
    if svc_model.coef_.ndim == 1 or svc_model.coef_.shape[0] == 1:  # Binary classification
        class_coefs = svc_model.coef_[0]  # Use the first (and only) row
    else:  # Multi-class
        class_coefs = svc_model.coef_[predicted_class]


    # Calculate feature contributions (absolute values)
    feature_contributions = transformed_instance[0] * class_coefs
    abs_feature_contributions = np.abs(feature_contributions)

    # Get indices of the top 5 most important features
    top_features_indices = np.argsort(abs_feature_contributions)[::-1][:5]

    # Decode the words for the top 5 features
    feature_names = vectorizer.get_feature_names_out()

    print("Top 5 Features Contributing to the Decision:")
    for idx in top_features_indices:
        feature_name = feature_names[idx]
        print(f"Feature {feature_name}: Value = {transformed_instance[0][idx]}, Contribution = {feature_contributions[idx]}, Absolute Contribution = {abs_feature_contributions[idx]}")

else:
    print("The loaded model is not a LinearSVC.")
# print(training_df['concatenated_text'].iloc[2])
print("Decision Value:", decision_value)

Pipeline steps: {'vec': TfidfVectorizer(max_df=0.9, max_features=500, min_df=0.05,
                tokenizer=<bound method PreprocessGCAT.CCR_Tokenizer of <complete_collision.PreprocessGCAT object at 0x7dd9d48be7e0>>), 'pca': PCA(n_components=np.int64(179)), 'classifier': LinearSVC(C=100, loss='hinge', random_state=123)}
Top 5 Features Contributing to the Decision:
Feature arriv: Value = -0.13889484109437505, Contribution = 1.4549575618044457, Absolute Contribution = 1.4549575618044457
Feature ae: Value = -0.1363225252044991, Contribution = -1.1542385348352369, Absolute Contribution = 1.1542385348352369
Feature dispatch: Value = -0.1304030065595847, Contribution = 1.0991420505516865, Absolute Contribution = 1.0991420505516865
Feature answer: Value = 0.15882128552050928, Contribution = 1.0543238574979454, Absolute Contribution = 1.0543238574979454
Feature age: Value = -0.09944091244593539, Contribution = -0.7917870023661849, Absolute Contribution = 0.7917870023661849
Decision Value: 0.1